In [1]:
import json
import random
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain.schema import HumanMessage

from Utils.Prompt import human_query_generation_agent

OPENAI_API_KEY = os.getenv('OPEN_AI_API')

def create_prompt():
    count = 0

    #day
    day_list = ['2 day','3 days', '4 days']
    day = random.choice(day_list)
    count += 1

    #price
    price_list = ["cheap budget","moderate budget","expensive budget"]
    price = random.choice(price_list)
    count += 1

    #attraction orientation
    attraction_orientation_list = ["family oriented","history oriented","activity oriented","nature oriented","food oriented","shopping oriented"]
    attraction_orientation = random.choice(attraction_orientation_list)
    count += 1

    #cuisine
    cuisine_list = ["US","Mexican","Irish","French","Italian","Greek","Indian","Chinese","Japanese","Korean","Vietnamese","Thai","Asian Fusion","Middle Eastern"]
    cuisine = random.choice(cuisine_list)
    count += 1

    #restaurant rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [1, 2, 3]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good flavor","good freshness","good service","good environment","good value"]
    restaurant_n = random.sample(n_list, coins)
    restaurant_n_str = ', '.join(restaurant_n)
    count += coins
    
    #hotel rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [1, 2, 3]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good quality","good location","good service","good safety"]
    hotel_n = random.sample(n_list, coins)
    hotel_n_str = ', '.join(hotel_n)
    count += coins

    #parse the prompt
    prompt = {
        "general":day + ", " + price,
        "attraction":attraction_orientation,
        "restaurant":cuisine + (', ' if len(restaurant_n) > 0 else "") + restaurant_n_str,
        "hotel":hotel_n_str
    }

    eval_info = {
        "day":[day],
        "price":[price],
        "attraction":[attraction_orientation],
        "cuisine":[cuisine],
        "restaurant":restaurant_n,
        "hotel":hotel_n,
        "preference_count":[count]
    }

    return prompt,eval_info

class QueryGenerationAgent():
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0,
                        model_name='gpt-4o-2024-11-20',
                        openai_api_key=OPENAI_API_KEY)
        
    def generateHumanQuery(self, input, generation_agent):
        content = generation_agent.format(input = input)
        print(content)
        request = self.llm.invoke([HumanMessage(content)]).content
        return request
    
if __name__ == "__main__":
    prompt_input,eval_info = create_prompt()
    print(prompt_input, eval_info)

    genAgent = QueryGenerationAgent()
    humanQuery = genAgent.generateHumanQuery(prompt_input,human_query_generation_agent)
    print(humanQuery)

{'general': '4 days, expensive budget', 'attraction': 'history oriented', 'restaurant': 'Greek, good environment', 'hotel': 'good quality'} {'day': ['4 days'], 'price': ['expensive budget'], 'attraction': ['history oriented'], 'cuisine': ['Greek'], 'restaurant': ['good environment'], 'hotel': ['good quality'], 'preference_count': [6]}

Craft a a human like query for a travel plan given the following information. The input includes details such as trip duration, budget type, attractions types that the traveler wants to visit, dining preferences that they want to try, and accommodation requirements. Make sure each pairs of key words, like good environment, good location, are mentioned specifically.

----- Example Starts -----
Input: 
- general: 2 days, moderate budget, 
- attraction: history oriented, 
- restaurants: French, good environment, 
- hotel: good quality, good location

Output:
I want to go for a 2-day trip with a moderate budget. I want to visit some history-oriented attracti